In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import gensim

# idee

NLP:
    - pos tagging per contare numero di aggettivi 
    - qualcosa per vedere se aggettivi sono positivi o negativi (direttamente sentiment analysis)?
    - rimuovere gli @ vs tenere solo i più frequenti 
    - rimuovere i link
    - pulire gli hashtag? 
    - word2vec distance per avere simili tweet assieme nello spazio vettoriale?
    - ridurre parole presenti? 
    - tfidf?
    
Processo:
    - clustering interno all'anno
    - clustering generale 
    - clustering con/senza sentiment meaning

In [ ]:
data = pd.read_json("dataWithTags.json")
data['created_at'] = pd.to_datetime(data.created_at, format="%Y-%m-%d %H:%M:%S")
data.sort_values(by=['created_at'], inplace=True, ascending=True)
data.reset_index(inplace=True,drop=True)
data

In [ ]:
data.drop(["in_reply_to_user_id_str","id_str","source"], inplace=True,axis=1) 

# Aggiungi dati per il tempo

In [ ]:
#exploit time
data['month'] = [d.month for d in data['created_at']]
data["year"] = [d.year for d in data["created_at"]]
data["hour"] = [d.hour for d in data["created_at"]]
data["week_year"] = [d.weekofyear for d in data["created_at"]]
data["date"] = [d.date() for d in data["created_at"]]

In [ ]:
temp = data.groupby(["year"])["year"].count()
plt.plot(temp)

annuncio campagna elettorale giugno 2015

In [ ]:
temp = pd.DataFrame({'count' : data.groupby( [ "year", "month"] ).size()}).reset_index()
#temp
#plotta nel corso dei mesi il numero di tweet per anni, devi prendere i dati per ogni anno e farli di lunghezza uguale come array e poi plotti 

In [ ]:
l = []
for year in temp["year"].unique().tolist():
    l1 = [0] * 12
    t = temp[temp["year"] == year]
    for idx,row in t.iterrows():
        l1[row["month"]-1] = row["count"]
    l.append(l1)
l

In [ ]:
leg = temp["year"].unique().tolist()
fig, ax = plt.subplots(figsize = (15, 10))
months = [x for x in range(0,12)]
for i in range(0,len(l)):
    plt.plot(months, l[i],label = leg[i])
plt.legend()

In [ ]:
data.drop("created_at", inplace=True,axis=1)
data.drop("is_retweet", inplace=True,axis=1)
data

# trasforma in label i valori delle date

In [ ]:
from sklearn.preprocessing import LabelEncoder

#trasforma la data in una label
labeller = LabelEncoder()
data["date"] = labeller.fit_transform(data["date"])

In [ ]:
data

# Stemming 


In [ ]:
stemmedTweets = []

for idx, row in data.iterrows():
    test = row["modded_text"].split()
    desc = ""
    for el in test:
        desc = desc + " " + stemmer.stem(el)
    stemmedTweets.append(desc)
temp = pd.Series(stemmedTweets)
data["stemmed"] = temp

In [ ]:
#remove stopwords
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')

data['stemmed'] = data['stemmed'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data.head()

In [ ]:
stemmed_words = pd.Series(' '.join(data['stemmed']).split()).value_counts()
print(len(stemmed_words))
stemmed_words

In [ ]:
stemmed_words1 = stemmed_words[stemmed_words > 10]
stemmed_words1

In [ ]:
non_stemmed = pd.Series(' '.join(data['modded_text']).split()).value_counts()
print(len(non_stemmed))

In [ ]:
data.columns

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import scipy.sparse as sp

data1 = data.drop(["text","modded_text"],axis=1)
data1 = data[data["year"] == 2018]
data2 = data1[["stemmed","date"]]

vectorizer = TfidfVectorizer(stop_words='english')

X = vectorizer.fit_transform(data2["stemmed"])

for i, col in enumerate(vectorizer.get_feature_names()):
    data2[col] = pd.SparseSeries(X[:, i].toarray().ravel(), fill_value=0)


#df1 = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())   
#df1
data2


#true_k = 100
#model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
#model.fit(X)


In [ ]:
model.inertia_

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

# Word2vec embedding

# tentativo di clustering su feature demmerda (inutile al chezzo ma vabe il codice è pronto)


data.drop(['created_at', 'text',"modded_text","id_str","is_retweet","in_reply_to_user_id_str","source"], axis=1,inplace=True) 


from sklearn import preprocessing
from sklearn.cluster import KMeans

x = data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.head()


kmeans = KMeans(n_clusters=2).fit(df)

sse_list = list()
max_k = 15
#plt.figure((12,6))
for k in range(2,max_k):
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10, max_iter=100)
    kmeans.fit(df)
    sse = kmeans.inertia_
    sse_list.append(sse)
plt.plot(range(2,max_k), sse_list)
plt.show()


kmeans = KMeans(n_clusters=40).fit(df)

hist, bins = np.histogram(kmeans.labels_, bins=range(0, len(set(kmeans.labels_)) + 1))
print('centers',kmeans.cluster_centers_)
print('labels', dict(zip(bins, hist)))
print('sse', kmeans.inertia_)